<a href="https://colab.research.google.com/github/Tommaso879/BigDataMining/blob/main/HW1_Ex1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u422-b05-1~22.04).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [3]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from pyspark.sql import SparkSession

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os

print(os.getcwd())  # This will print the current working directory
print(os.listdir())  # This will list all files and directories

os.chdir('/content/drive/MyDrive/Data/HW1/EX1')

/content
['.config', 'drive', 'sample_data']


In [6]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
#sc.stop()  #required if a context already exists and you want to re-run he cell
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()
# create the Spark Context
sc = spark.sparkContext

data = spark.read.text("soc-LiveJournal1Adj.txt")

In [7]:
spark

Load the datasets:

In [8]:
data.printSchema()

root
 |-- value: string (nullable = true)



Check the schema:

In [9]:
data.take(3)

[Row(value='0\t1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94'),
 Row(value='1\t0,5,20,135,2409,8715,8932,10623,12347,12846,13840,13845,14005,20075,21556,22939,23520,28193,29724,29791,29826,30691,31232,31435,32317,32489,34394,35589,35605,35606,35613,35633,35648,35678,38737,43447,44846,44887,49226,49985,623,629,4999,6156,13912,14248,15190,17636,19217,20074,27536,29481,29726,29767,30257,33060,34250,34280,34392,34406,34418,34420,34439,34450,34651,45054,49592'),
 Row(value='2\t0,117,135,1220,2755,12453,24539,24714,41456,45046,49927,6893,13795,16659,32828,41878')]

In [10]:
data.show()

+--------------------+
|               value|
+--------------------+
|0\t1,2,3,4,5,6,7,...|
|1\t0,5,20,135,240...|
|2\t0,117,135,1220...|
|3\t0,12,41,55,153...|
|4\t0,8,14,15,18,2...|
|5\t0,1,20,2022,22...|
|6\t0,21,98,2203,3...|
|7\t0,31993,40218,...|
|8\t0,4,38,46,72,8...|
|9\t0,6085,18972,1...|
|10\t0,12,16,30,60...|
|11\t0,1754,6027,7...|
|12\t0,3,10,16,29,...|
|13\t0,12584,32064...|
|14\t0,4,19,19079,...|
|       15\t0,4,27,80|
|16\t0,10,12,18,30...|
|17\t0,19,26,28,95...|
|18\t0,4,16,30,89,...|
|19\t0,14,17,439,1...|
+--------------------+
only showing top 20 rows



In [11]:
print("in total there are {0} people" .format(data.count()))

in total there are 49995 people


In [12]:
split_df = data.select(
    split(data['value'], '\t').getItem(0).alias('user'),
    split(data['value'], '\t').getItem(1).alias('friends')
)


split_df.show(truncate=False)


+----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|user|friends                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [13]:
# Split friends string by comma and explode into individual rows
exploded_df = split_df.withColumn("friend", explode(split(col("friends"), ",")))

# Cast user and friend columns to integer types
exploded_df = exploded_df.select(col("user").cast("int"), col("friend").cast("int"))

# Show the exploded DataFrame
exploded_df.show(truncate=False)

+----+------+
|user|friend|
+----+------+
|0   |1     |
|0   |2     |
|0   |3     |
|0   |4     |
|0   |5     |
|0   |6     |
|0   |7     |
|0   |8     |
|0   |9     |
|0   |10    |
|0   |11    |
|0   |12    |
|0   |13    |
|0   |14    |
|0   |15    |
|0   |16    |
|0   |17    |
|0   |18    |
|0   |19    |
|0   |20    |
+----+------+
only showing top 20 rows



In [14]:
# Self-join to find friends of friends
friends_of_friends = exploded_df.alias("df1").join(
    exploded_df.alias("df2"),
    col("df1.friend") == col("df2.user")
).select(
    col("df1.user").alias("user"),
    col("df2.friend").alias("friend_of_friend")
)

# Show friends of friends
friends_of_friends.show(truncate=False)

+----+----------------+
|user|friend_of_friend|
+----+----------------+
|0   |49592           |
|0   |45054           |
|0   |34651           |
|0   |34450           |
|0   |34439           |
|0   |34420           |
|0   |34418           |
|0   |34406           |
|0   |34392           |
|0   |34280           |
|0   |34250           |
|0   |33060           |
|0   |30257           |
|0   |29767           |
|0   |29726           |
|0   |29481           |
|0   |27536           |
|0   |20074           |
|0   |19217           |
|0   |17636           |
+----+----------------+
only showing top 20 rows



In [15]:
# Filter out the friends who are already direct friends
filtered_result = friends_of_friends.join(
    exploded_df,
    (friends_of_friends.user == exploded_df.user) & (friends_of_friends.friend_of_friend == exploded_df.friend),
    how="left_anti"
)

# Show the final result
filtered_result.show(truncate=False)

+----+----------------+
|user|friend_of_friend|
+----+----------------+
|0   |49592           |
|0   |45054           |
|0   |34651           |
|0   |34450           |
|0   |34439           |
|0   |34420           |
|0   |34418           |
|0   |34406           |
|0   |34392           |
|0   |34280           |
|0   |34250           |
|0   |33060           |
|0   |30257           |
|0   |29767           |
|0   |29726           |
|0   |29481           |
|0   |27536           |
|0   |20074           |
|0   |19217           |
|0   |17636           |
+----+----------------+
only showing top 20 rows



In [16]:
filtered_result = filtered_result.filter(col("user") != col("friend_of_friend"))

In [17]:
# Check for duplicates by grouping by user and friend_of_friend
duplicates_check = filtered_result.groupBy("user", "friend_of_friend").count()

# Show entries with more than one occurrence (which means duplicates exist)
duplicates_check.filter(col("count") > 1).show(truncate=False)


+----+----------------+-----+
|user|friend_of_friend|count|
+----+----------------+-----+
|0   |337             |3    |
|0   |343             |2    |
|0   |352             |3    |
|0   |543             |2    |
|0   |623             |2    |
|0   |1532            |3    |
|0   |2644            |2    |
|0   |6027            |2    |
|0   |6169            |2    |
|0   |7284            |2    |
|0   |9745            |2    |
|0   |10506           |2    |
|0   |12143           |3    |
|0   |12561           |3    |
|0   |12636           |2    |
|0   |13285           |2    |
|0   |15520           |2    |
|0   |16514           |2    |
|0   |18591           |4    |
|0   |19051           |2    |
+----+----------------+-----+
only showing top 20 rows



In [18]:
requesedUserID = input("Enter the user ID: ")
print("here's the result for user ", requesedUserID)
result = duplicates_check.select("user", "friend_of_friend", "count").filter(col("user") == requesedUserID)
result = result.sort(col("count").desc(), col("friend_of_friend").asc())
result.limit(10).show(truncate=False)

expected_result = [[27552,7785,27573,27574,27589,27590,27600,27617,27620,27667]]
print("here's the expected result")
print(expected_result)


Enter the user ID: 11
here's the result for user  11
+----+----------------+-----+
|user|friend_of_friend|count|
+----+----------------+-----+
|11  |27552           |4    |
|11  |7785            |3    |
|11  |27573           |3    |
|11  |27574           |3    |
|11  |27589           |3    |
|11  |27590           |3    |
|11  |27600           |3    |
|11  |27617           |3    |
|11  |27620           |3    |
|11  |27667           |3    |
+----+----------------+-----+

here's the expected result
[[27552, 7785, 27573, 27574, 27589, 27590, 27600, 27617, 27620, 27667]]


In [19]:
wanted_values = [8997, 2791, 4985, 8961, 4049, 5060, 739, 1724, 9550, 3151]

for i in wanted_values:
  print("here's the result for user ", i)
  result = duplicates_check.select("user", "friend_of_friend", "count").filter(col("user") == i)
  result = result.sort(col("count").desc(), col("friend_of_friend").asc())
  result.limit(10).show(truncate=False)

here's the result for user  8997
+----+----------------+-----+
|user|friend_of_friend|count|
+----+----------------+-----+
|8997|8998            |4    |
|8997|8987            |3    |
|8997|8992            |2    |
|8997|9001            |2    |
|8997|9003            |2    |
|8997|9009            |2    |
|8997|4849            |1    |
|8997|7174            |1    |
|8997|7279            |1    |
|8997|7364            |1    |
+----+----------------+-----+

here's the result for user  2791
+----+----------------+-----+
|user|friend_of_friend|count|
+----+----------------+-----+
|2791|21185           |5    |
|2791|8783            |4    |
|2791|13280           |4    |
|2791|18359           |4    |
|2791|18363           |4    |
|2791|23667           |4    |
|2791|35740           |4    |
|2791|2204            |3    |
|2791|2786            |3    |
|2791|5996            |3    |
+----+----------------+-----+

here's the result for user  4985
+----+----------------+-----+
|user|friend_of_friend|count|